In [3]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pathlib import Path
from tratamento import trata_acomph, a0_a1, tipo3

In [4]:
def get_prod():
    loc = Path('../ex_csv/produtibilidades/prod.csv')
    prod = pd.read_csv(loc, index_col=0)
    for i,row in prod.head(154).iterrows():
        row['prod'] = row['prod'].replace(",",".")
        row['prod'] = float(row['prod'])
    return prod

In [5]:
def importa_arquivos():
    acomph = trata_acomph.get_csv()
    produtibilidade = get_prod()
    a0, a1 = a0_a1.get_csv()
    local = Path('../ex_csv')
    local_post = local / 'postos.csv'
    postos = pd.read_csv(local_post, index_col=0)
    return acomph, a0, a1, postos, produtibilidade

In [6]:
def get_datas():
    meses = []
    for i in range(1,31):
        meses = [((date.today()-timedelta(days=i)).month)] + meses
    return meses

In [7]:
def trata_vazoes_base(acomph, a1):
    vazoes_base = acomph.loc[a1.iloc[:,0], :]
    vazoes_base.reset_index(inplace=True)
    base = a1.reset_index()
    indice = base.loc[:,'posto']
    vazoes_base.insert(0, "ind", indice, allow_duplicates=True)
    vazoes_base.drop('posto', axis=1, inplace=True)
    vazoes_base.rename(columns={'ind':'posto'}, inplace=True)
    vazoes_base.set_index('posto', inplace=True)
    vazoes_base.dropna(inplace=True)
    return vazoes_base

In [8]:
def cria_ena():
    acomph, a0, a1, postos, produtibilidades = importa_arquivos()
    ind = acomph.head(155).index
    col = (acomph.T).head(30).index
    ena = pd.DataFrame(0,index = ind, columns = col)
    ena = ena.add(ena_tipo_0(ena,acomph,produtibilidades,postos), fill_value = 0)
    ena = ena.add(ena_tipo_1(ena,acomph,produtibilidades,postos,a0,a1), fill_value = 0)
    ena = ena_tipo_3(ena)
    return ena

In [9]:
def ena_tipo_0(ena, acomph, produtibilidades, postos):
    vazoes_tipo0 = acomph.join(postos.query('tipo==0'),
                               on = 'posto', how = 'inner')
    vazoes_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo0 = produtibilidades.join(postos.query('tipo==0'),
                                                  on = 'posto', how = 'inner')
    produtibilidade_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'], axis=1, inplace=True)
    for i in range(30):
        ena_dia = vazoes_tipo0.iloc[:,i].multiply(produtibilidade_tipo0['prod'], fill_value=0)
        ena.iloc[:,i] = ena.iloc[:,i].add(ena_dia, fill_value = 0)
    return ena

In [10]:
def ena_tipo_1(ena, acomph, produtibilidades, postos, a0, a1):
    meses = get_datas()
    vazoes_tipo1 = acomph.join(postos.query('tipo==1'),
                               on = 'posto', how = 'inner')
    vazoes_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo1 = produtibilidades.join(postos.query('tipo==1'),
                                                 on='posto', how='inner')
    produtibilidade_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                              axis=1, inplace=True)
    vazoes_base = trata_vazoes_base(acomph, a1)
    for i in range(30):
        A0 = a0.iloc[:,meses[i]]
        A1 = a1.iloc[:,meses[i]]
        X = vazoes_base.iloc[:,i]
        Y = A0 + (A1*X)
        ena_dia = Y.multiply(produtibilidade_tipo1['prod'], fill_value=0)
        ena.iloc[:,i] = ena.iloc[:,i].add(ena_dia, fill_value = 0)
    return ena

In [11]:
def ena_tipo_3(ena):
    ena.append([tipo3.posto_126, tipo3.posto_127, tipo3.posto_131, tipo3.posto_132,
                tipo3.posto_176, tipo3.posto_285, tipo3.posto_292, tipo3.posto_298, 
                tipo3.posto_299, tipo3.posto_303, tipo3.posto_304, tipo3.posto_306,
                tipo3.posto_315, tipo3.posto_316, tipo3.posto_317, tipo3.posto_318,
                tipo3.posto_37, tipo3.posto_38, tipo3.posto_39, tipo3.posto_40,
                tipo3.posto_42, tipo3.posto_43, tipo3.posto_45, tipo3.posto_46,
                tipo3.posto_66, tipo3.posto_75])
    return ena

In [12]:
cria_ena()

/home/mazucanti/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,...,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30
posto,,,,,,,,,,,,,,,,,,,,,
1,34.0226,31.8249,29.3892,27.4315,26.3876,25.7464,25.674,30.1912,41.257,46.5398,...,26.2781,25.9849,25.7472,25.2975,24.6477,26.2436,27.8336,26.5372,27.0992,27.2381
6,898.063,786.776,699.237,640.883,589.23,556.378,535.027,522.596,527.021,590.577,...,496.616,500.943,510.953,514.388,517.325,525.811,539.783,557.167,564.108,601.376
7,215085,188432,167466,153491,141120,133252,128138,125161,126221,141442,...,118939,119975,122372,123195,123899,125931,129277,133441,135103,144029
8,369881,324045,287991,263957,242683,229153,220359,215239,217061,243238,...,204539,206321,210443,211858,213068,216563,222318,229478,232336,247686
9,268109,234885,208751,191330,175910,166102,159728,156017,157337,176312,...,148260,149552,152541,153566,154443,156976,161148,166337,168410,179536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,188.315,215.979,191.65,153.665,139.587,143.133,143.097,202.112,223.001,247.436,...,285597,285597,271695,271.73,254.281,261.409,295.883,311.629,261268,271695
294,158.389,160.92,164.697,168.956,173.469,178.784,184.466,190.571,195.314,199.128,...,194.269,196.135,195.457,193.747,189.116,185.776,181.92,178.725,176.853,176.489
295,261.785,259.578,259.717,262.446,265.001,262.625,255.896,250.492,245.854,243.726,...,284.333,274.322,263.291,253.463,250.639,247.107,246.681,248.278,245.21,244.191
